In [1]:
import numpy as np
#import os

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

from astropy.io import fits

from setup import *
from FullFrame import FullFrame

##--------------------------------------------------------------------------##
##             NECESSARY USER INPUTS BELOW....                              ##
##--------------------------------------------------------------------------##

##################
###   WASP4b   ###
##################
# obs_date='ut20150819'                           #observation date
# obj_name='Wasp4'                                #object name   
# midtime=['2015-08-19T07:43:00.00']                 #time of midtransit, WAsp-4b

# obj_skip=[2,6,9]
# binnx=1
# binny=1

###################
###   WASP52b   ###
###################
#obs_date='ut20160811'
#obs_date='ut20160922'
#obj_name='Wasp52'

#midtime=['2016-08-11T04:51:00']               #time of midtransit, Wasp-52b, Transit1
#midtime=['2016-09-22T04:44:00']               #time of midtransit, Wasp-52b, Transit2

#obj_skip=[1,4,6,7]
#binnx=1
#binny=1
#
##################
###   Hats8b   ###
##################
obs_date='ut20170723'                           #observation date
#obs_date='ut20170828'
obj_name='Hats8'                                #object name  

midtime=['2017-07-24T05:28:01.056']                 #time of midtransit, Hats-8b, Transit 1
#midtime=['2017-08-29T01:36:04.608']                 #time of midtransit, Hats-8b Transit 2

obj_skip=[]
binnx=1 #(night 1)
binny=1

#binnx=2  #(night 2)
#binny=2

##################
### Wasp 67b  ###
##################
# obs_date='ut20180521'
# obj_name='Wasp67'
# midtime=['2018-05-22T06:54.00']

#obj_skip=[8,10]
#binnx=2
#binny=2

##################
### HATS-35b  ###
##################
# obs_date='ut20180524'
# obj_name='Hats35'
# midtime=['2018-05-25T06:51.00']

# obj_skip=[]
# binnx=2
# binny=2

##################
###   WD1145   ###
##################
# obs_date='ut20180521'                           #observation date

# obj_name='WD1145'                                #object name   
# midtime=['2018-05-21T23:00:00.00']                 #time of midtransit, WD1145

# obj_skip=[10]
# binnx=2
# binny=2
###########################

path='/Volumes/ermay_ext/Magellan_IMACS/'+obs_date+'/'
#path='/Users/ermay/Desktop/Magellan/'+obs_date+'/'
data_path=path+obj_name+'_Spec/'      #path to where your data is saved
flat_path=path+obj_name+'_Flats/'     #path to where your SLIT flats are saved
DARKS=path+'Darks'     #path to where your darks are saved
FLATS=path+'Full_Flats/'             #path to where the full field flats are saved 
WAVE=path+obj_name+'_Arcs/'

#SAVEPATH='SaveData/'+obj_name+'/'
SAVEPATH=path+'SaveData_'+obj_name+'_ap5/'
#SAVEPATH='/Users/ermay/Desktop/Checking_Outliers_ML/'

wavelength_path=path+'WaveSolution_'+obj_name+'/'

extray=400/binny #number of extra pixels to extract per side in y direction
extrax=80/binnx  #number on the side to find wavelength strip, only for nights with a single mask

#### CHECK BINNING IN SETUP.PY ####
#### if 1x1 binning, xpixels,ypixels should be 2048,4096
#### if 2x2 binning, xpixels,ypixels should be 1024,2048
print xpixels/binnx, ypixels/binny

2048 4096


In [10]:
width=100
avg=False

nwalkers=1000
burnin=3000
nsteps=75000

color=1
cont=True

In [11]:
#####################################################                                                                                           
#####################################################                                                                                           
##        Fitting of lightcurves using BATMAN      ##                                                                                           
##  ---------------------------------------------  ##                                                                                           
##  reads in .npz files from Lightcurve_Script.py  ##                                                                                           
##     See README.txt in Magellan/pipeline         ##                                                                                           
#####################################################                                                                                           
#####################################################   

import numpy as np
np.seterr(all='ignore')

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import batman

import emcee
import corner

#import astropy
#from astropy.time import Time
#import pyfits                                                                                                                                  

#import scipy
from scipy.interpolate import interp1d
#from scipy.signal import medfilt

from datetime import datetime
startTime1=datetime.now()

from outlier_removal import outlierr

import sys
print (sys.argv)
 

sys.path.insert(0,SAVEPATH) 
from SystemCons import *

print per, rp, sma

#width=200


#################################
t=np.load(SAVEPATH+'Obs_times.npz')['times']
n_exp=len(t)
fulltime=np.load(SAVEPATH+'Obs_times.npz')['times']
t_z=np.load(SAVEPATH+'LCwhite_br.npz')['avt']


if avg==True:
    t=np.load(SAVEPATH+'LCwhite_br.npz')['avt']
    n_exp=len(t)

    lc_data_white=np.load(SAVEPATH+'LCwhite_br.npz')['avf']
    lc_data_binns=np.load(SAVEPATH+'LC_bins_br_'+str(int(width))+'.npz')['avf']
    yerr_white=np.load(SAVEPATH+'LCwhite_br.npz')['rmse']
    yerr_binns=np.load(SAVEPATH+'LC_bins_br_'+str(int(width))+'.npz')['rmse']
    #yerr_binns=np.load(SAVEPATH1+'Binned_Data_'+str(int(width))+'.npz')['bin_counts']


else:   
    t=np.load(SAVEPATH+'Obs_times.npz')['times']
    n_exp=len(t)

    lc_data_white=np.load(SAVEPATH+'LCwhite_br.npz')['data']
    lc_data_binns=np.load(SAVEPATH+'LC_bins_br_'+str(int(width))+'.npz')['data']
    #yerr_white=np.load(SAVEPATH+'LCwhite_br.npz')['rmse']
    #yerr_binns=np.load(SAVEPATH+'LC_bins_br_'+str(int(width))+'.npz')['rmse']
    yerr_white=1./np.sqrt((np.load(SAVEPATH+'Binned_Data_White.npz')['bin_counts'])[0,:,0])
    yerr_binns=1./np.sqrt((np.load(SAVEPATH+'Binned_Data_'+str(int(width))+'.npz')['bin_counts'])[0,:,:])

    
bin_ctr=np.load(SAVEPATH+'LC_bins_br_'+str(int(width))+'.npz')['bin_ctr']
print lc_data_white.shape
print len(lc_data_white)

####################################################                                                                                            
norm=matplotlib.colors.Normalize(vmin=np.min(bin_ctr),vmax=np.max(bin_ctr))                                                                                                                 
colors=matplotlib.cm.Spectral_r
scal_m=matplotlib.cm.ScalarMappable(cmap=colors,norm=norm)
scal_m.set_array([])

####################################################                                                                                            
#              LIMB DARKENING FUNCION              #                                                                                            
####################################################                                                                                            
# U B V R I J H K                                                                                                                               

limb_dark='quadratic'

wave=[3640,4450,5510,6580,8060,12200]#,16300,21900]                                                                                             

q0_func=interp1d(wave,q0)
q1_func=interp1d(wave,q1)

####################################################                                                                                            
#              LIGHT CURVE FUNCTION                #                                                                                            
####################################################                                                                                            
def lc_func(t,t0,per,rp,a,inc,ecc,w,u,limb_dark):
    params=batman.TransitParams()
    params.t0=t0
    params.per=per
    params.rp=rp
    params.a=a
    params.inc=inc
    params.ecc=ecc
    params.w=w
    params.u=u
    params.limb_dark=limb_dark

    model=batman.TransitModel(params,t)
    return params, model

####################################################                                                                                            
#     Prior, Likelihood, Posterior Functions       #                                                                                            
####################################################                                                                                            
def loggaus(mean,sig,param):
    return -np.log(np.sqrt(2*np.pi*sig**2.))-((param-mean)**2.)/(2*sig**2.)

def lnprior(theta,color):
    if color==0:
        t0_f,per_f,rp_f,inc_f,c1_f,c2_f=theta
        if c1_f>-2.0 and c1_f<2.0 and c2_f>-2.0 and c1_f<2.0 and t0_f>t0dn and t0_f<t0up and rp_f>0.0 and rp_f<1.0:
            return loggaus(per,per_e,per_f)+loggaus(inc,inc_e,inc_f)
        #if t0_f>t0dn and t0_f<t0up and rp_f>rpdn and rp_f<rpup and np.abs(c2_f-c2)<0.1 and np.abs((c2_f-c2)-(c1_f-c1))<0.05:
        #    return loggaus(per,per_e,per_f)+loggaus(inc,inc_e,inc_f)
        else:
            return -np.inf
    if color==1:
        rp_f,c1_f,c2_f=theta
        if rp_f<rpup and rp_f>rpdn and np.abs(c2_f-c2)<0.1 and np.abs((c2_f-c2)-(c1_f-c1))<0.05:
            return 0.0
        else:
            return -np.inf

def lnlike(theta,params,model,t,flux,err,color):
    if color==0:
        params.t0,params.per,params.rp,params.inc,c1_f,c2_f=theta
        params.u=[c1_f,c2_f]
        Ms_s=np.random.normal(loc=Ms,scale=Ms_e)
        Rs_s=np.random.normal(loc=Rs,scale=Rs_e)
        smacm=(((params.per*24.*60.*60.)**2.*Grav*Ms_s)/(4*np.pi**2.))**(1./3.)
        params.a=smacm/Rs_s
        fit=model.light_curve(params)
        residuals=flux-fit
        return -0.5*(np.nansum((residuals/err)**2))
    if color==1:
        params.rp,c1_f,c2_f=theta
        params.u=[c1_f,c2_f]
        fit=model.light_curve(params)
        residuals=flux-fit
        return -0.5*(np.nansum((residuals/err)**2))


def lnprob(theta,params,model,t,flux,err,color):
    prior=lnprior(theta,color)
    if not np.isfinite(prior):
        return -np.inf
    post=prior+lnlike(theta,params,model,t,flux,err,color)
    return post

####################################################                                                                                            
#     RUN MCMC CODE      #                                                                                            
####################################################   

def runmcmc(nwalkers,burnin,nsteps,pos0,params,model,time,data,error,color):
    sampler=emcee.EnsembleSampler(nwalkers,ndim,lnprob,a=2.0, args=(params,m,t,data,error,color))
    sampler.reset()
    
    Output=open(SAVEPATH+'UNC_Fits_'+str(int(width))+'/Progress.txt','a')
    Output.write('{0} \n'.format('     -->> Running Burn-in...'))
                                                                                                                        
    time1=datetime.now()
    p0,test1,test2=sampler.run_mcmc(pos0,burnin)
    time2=datetime.now()
    Output.write('{0} {1} \n'.format('           Time to Run: ', time2-time1))
    sampler.reset()
                                                                                                                          

    Output.write('{0} \n'.format('     -->> Running Chain...'))
    Output.close()
    for i, result in enumerate(sampler.sample(p0,iterations=nsteps)):
        if (i+1)%(nsteps/10) ==0:
            Output=open(SAVEPATH+'UNC_Fits_'+str(int(width))+'/Progress.txt','a')
            Output.write('{0} {1} {2} \n'.format(("            {0:5.1%}".format(float(i+1) / nsteps)),'          ', datetime.now()))
            Output.close()

    Output=open(SAVEPATH+'UNC_Fits_'+str(int(width))+'/Progress.txt','a')
    Output.write('{0} {1} \n'.format('     -->> Mean Acceptance Fraction: ', np.mean(sampler.acceptance_fraction)))
    Output.close()
    
    samples=sampler.chain[:,:,:].reshape((-1,pos0.shape[1]))
    
    return samples,sampler

    

#####################
#nwalkers=500
#burnin=500
#nsteps=3000

Output=open(SAVEPATH+'UNC_Fits_'+str(int(width))+'/Progress.txt','w')

#color=0
if color==0:
    Output=open(SAVEPATH+'UNC_Fits_'+str(int(width))+'/Progress.txt','a')
    Output.write('{0}\n'.format('>>>>>>>>>> WHITE LIGHT CURVE <<<<<<<<<<'))
    print('>>>>>>>>>> WHITE LIGHT CURVE <<<<<<<<<<')
    
    print('-----------> outliers')
    lc_data_white=outlierr(lc_data_white,5,3)
    #lc_data_white=outlierr(lc_data_white,5,3)
    
    #if np.nan(yerr_white)==0.0:
    #    yerr_white=np.ones_like(yerr_white)*10**-3.
    
    c1=q0_func(6000)
    c2=q1_func(6000)
    u=[c1,c2]
                                                                                                                          
    params,m=lc_func(t,t0,per,rp,sma,inc,ecc,w,u,limb_dark)
    
    initial=np.array([t0,per,rp,inc,c1,c2])
    ndim=len(initial)
    
    Output.write('{0} {1} \n'.format('     -->> Initial Guess: ', initial))
    Output.close()

    t0_arr=np.linspace(t0dn,t0up,10**6.)
    per_arr=np.linspace(per-5*per_e,per+5*per_e,10**6.)
    rp_arr=np.linspace(0.0,0.3,10**6.)                                                                                              
    inc_arr=np.linspace(inc-5*inc_e,inc+5*inc_e,10**6.)
    c1_arr=np.linspace(-2.0,2.0,10**6.)
    c2_arr=np.linspace(-2.0,2.0,10**6.)  

    pos0=np.empty([nwalkers,ndim])
    for i in range(nwalkers):
        pos0[i,:]=np.array([np.random.choice(t0_arr),np.random.choice(per_arr),np.random.choice(rp_arr),np.random.choice(inc_arr),np.random.choice(c1_arr),np.random.choice(c2_arr)])
        
    runwhite,whitechain=runmcmc(nwalkers,burnin,nsteps,pos0,params,m,t,lc_data_white,yerr_white,color)
    
    t0o,pero,rpo,inco,c1o,c2o=map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),zip(*np.percentile(runwhite, [16, 50, 84], axis=0)))
    t0,per,rp,inc,c1,c2=t0o[0],pero[0],rpo[0],inco[0],c1o[0],c2o[0]
    u=[c1,c2]
    sma=(((per*24.*60.*60.)**2.*Grav*Ms)/(4*np.pi**2.))**(1./3.)
    sma=sma/Rs

    params,m=lc_func(t,t0,per,rp,sma,inc,ecc,w,u,limb_dark)
    paramsp,mp=lc_func(fulltime,t0,per,rp,sma,inc,ecc,w,u,limb_dark)
    paramsz,mz=lc_func(t_z,t0,per,rp,sma,inc,ecc,w,u,limb_dark)
        
    fitlightcurve=m.light_curve(params)
    fitlightcurvep=mp.light_curve(paramsp)
    fitlightcurvez=mz.light_curve(paramsz)
    
    residuals=(fitlightcurve-lc_data_white)*10**6.
    residuals=(fitlightcurvep-lc_data_white)*10**6.
    res_white=residuals
    
    chi2=np.nansum(np.abs(residuals/10**6.)**2.)

    Output=open(SAVEPATH+'UNC_Fits_'+str(int(width))+'/Progress.txt','a')
    Output.write('{0} \n'.format('     -->> Best Fit Params'))
    Output.write('{0} {1} {2} {3} \n'.format('          t0  : ', t0,  np.round(t0o[1],5),np.round(t0o[2],5)))
    Output.write('{0} {1} {2} {3} \n'.format('          per : ', per, np.round(pero[1],10),np.round(pero[2],10)))
    Output.write('{0} {1} {2} {3} \n'.format('          rp  : ', rp,  np.round(rpo[1],5),np.round(rpo[2],5)))
    Output.write('{0} {1} \n'.format('          a   : ', sma))#, np.round(smao[1],5),np.round(smao[2],5)))                                      
    Output.write('{0} {1} {2} {3} \n'.format('          inc : ', inc, np.round(inco[1],5),np.round(inco[2],5)))
    Output.write('{0} {1} {2} {3} \n'.format('          c1  : ', c1,  np.round(c1o[1],5),np.round(c1o[2],5)))
    Output.write('{0} {1} {2} {3} \n'.format('          c2  : ', c2,  np.round(c2o[1],5),np.round(c2o[2],5)))
    Output.write('{0} {1} \n'.format('  chi-squared : ', chi2))
    Output.close()

    plt.figure(1)                                                                                                                                 
    plt.plot(t,lc_data_white,'.',markersize=10,markeredgecolor='black',markerfacecolor='grey')
    plt.plot(fulltime,fitlightcurvep,'-',color='black')
    plt.ylim(np.nanmin(fitlightcurvep)-0.01,1.01)
    plt.figtext(0.15,0.15,'$\chi^2$ = '+str(chi2))
#    plt.figtext(0.55,0.60, str(int(bin_wav[b]))+' $\AA$',fontsize=25,color=scal_m.to_rgba(bin_wav[b]))                                         
    plt.figtext(0.55,0.80, 'White Light', fontsize=25,color='grey')
    plt.savefig(SAVEPATH+'UNC_Fit_Orbit_LC_white.png')
                
    t0min=np.max([t0dn,t0-5*t0o[2]])
    t0max=np.min([t0up,t0+5*t0o[1]])
    rpmin=np.max([rpdn,rp-5*rpo[2]])
    rpmax=np.min([rpup,rp+5*rpo[1]])
    
    plt.figure(2)
    corner.corner(runwhite,labels=['t0','per','rp','inc','c1','c2'],truths=[t0,per,rp,inc,c1,c2],range=([t0min,t0max],[per-5*pero[2],per+5*pero[1]],[rpmin,rpmax],[inc-5*inco[2],inc+5*inco[1]],[c1-5*c1o[2],c1+5*c1o[1]],[c2-5*c2o[2],c2+5*c2o[1]]))
    plt.savefig(SAVEPATH+'UNC_CornerPlot_white.png')

    Output=open(SAVEPATH+'UNC_Fits_'+str(int(width))+'/Progress.txt','a')
    Output.write('{0}\n'.format(' '))
    Output.write('{0} {1} \n'.format('            TIME TO RUN: ', datetime.now() - startTime1))
    Output.write('{0}\n'.format(' '))
    Output.close()

    params=np.array([params.t0,params.per,params.rp,params.a,params.inc,params.u[0],params.u[1]])
    paramserr=np.array([[t0o[1],t0o[2]],[pero[1],pero[2]],[rpo[1],rpo[2]],[inco[1],inco[2]]])

#     np.savez_compressed(SAVEPATH+'UNC_LightCurve_fits_white.npz',results=whitechain,params=params,paramserr=paramserr,
#                         lightcurve_fit=fitlightcurvep,lcfitz=fitlightcurvez,residuals=residuals)
    np.savez_compressed(SAVEPATH+'UNC_LightCurve_fits_white.npz',params=params,paramserr=paramserr,
                        lightcurve_fit=fitlightcurvep,lcfitz=fitlightcurvez,residuals=residuals)
    
    del whitechain
    del runwhite
    if cont==True:
        color=1

if color==1:
    save_rp=np.empty([len(bin_ctr)])*np.nan
    save_rp_e=np.empty([len(bin_ctr),2])*np.nan
    save_ld=np.empty([len(bin_ctr),2])*np.nan
    save_ld_e=np.empty([len(bin_ctr),4])*np.nan

    white_params=np.load(SAVEPATH+'UNC_LightCurve_fits_white.npz')['params']
    t0,per,rp,sma,inc,u0,u1=white_params
    u=np.array([u0,u1])
    params,m=lc_func(t,t0,per,rp,sma,inc,ecc,w,u,limb_dark)
    #params.t0,params.per,params.rp,params.a,params.inc,params.u[0],params.u[1]=white_params
    plt.figure(101,figsize=(4,4))
    plt.plot(t,m.light_curve(params),'-',color='grey')
    plt.plot(t,lc_data_white,'.',color='red',markersize=8)
    plt.show(block=False)
    
    for b in range(0,len(bin_ctr)):
        Output=open(SAVEPATH+'UNC_Fits_'+str(int(width))+'/Progress.txt','a')
        Output.write('{0} {1} {2}\n'.format('>>>>>>>>>> WAVELENGTH BIN: ' ,bin_ctr[b], ' <<<<<<<<<<'))
        print('>>>>>>>>>> WAVELENGTH BIN: ' ,bin_ctr[b], ' <<<<<<<<<<')
        if np.isnan(lc_data_binns[0,b]):
            continue
            
        plt.figure(201,figsize=(4,4))
        plt.plot(t,lc_data_binns[:,b],'.',markersize=8,markeredgecolor='black',markerfacecolor=scal_m.to_rgba(bin_ctr[b]))
        plt.show(block=False)
        
        print('-----------> outliers')
        lc_data_binns[:,b]=outlierr(lc_data_binns[:,b],5,3)
        #lc_data_binns[:,b]=outlierr(lc_data_binns[:,b],5,3)
        
        plt.figure(202,figsize=(4,4))
        plt.plot(t,lc_data_binns[:,b],'.',markersize=8,markeredgecolor='black',markerfacecolor=scal_m.to_rgba(bin_ctr[b]))
        plt.show(block=False)
                
        #lc_data_binns[:,b]=lc_data_binns[:,b]+res_white*10**-6
        
#        Output.write('{0} {1} \n'.format(' -->> WAVLENGTH CENTER = ', bin_ctr[b]))
    

        #if np.nanmax(yerr_binns[:,b])==0.0:
        #    hold=np.ones_like(yerr_binns[:,b])*10**-3.
        #    yerr_binns[:,b]=hold
    
        c1=q0_func(bin_ctr[b])
        c2=q1_func(bin_ctr[b])
        u=[c1,c2]
                                                                                                                          
        params,m=lc_func(t,t0,per,rp,sma,inc,ecc,w,u,limb_dark)
    
        initial=np.array([rp,c1,c2])
        ndim=len(initial)
    
        Output.write('{0} {1} \n'.format('     -->> Initial Guess: ', initial))
        Output.close()

        rp_arr=np.linspace(0.0,0.3,10**6.)                                                                                              
        c1_arr=np.linspace(-2.0,2.0,10**6.)
        c2_arr=np.linspace(-2.0,2.0,10**6.)  

        pos0=np.empty([nwalkers,ndim])
        for i in range(nwalkers):
            pos0[i,:]=np.array([np.random.choice(rp_arr),np.random.choice(c1_arr),np.random.choice(c2_arr)])
        
        runlam,lamchain=runmcmc(nwalkers,burnin,nsteps,pos0,params,m,t,lc_data_binns[:,b],yerr_binns[:,b],color)
    
        rpo,c1o,c2o=map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),zip(*np.percentile(runlam, [16, 50, 84], axis=0)))
        rp,c1,c2=rpo[0],c1o[0],c2o[0]
        u=[c1,c2]

        params,m=lc_func(t,t0,per,rp,sma,inc,ecc,w,u,limb_dark)
        paramsp,mp=lc_func(fulltime,t0,per,rp,sma,inc,ecc,w,u,limb_dark)
        paramsz,mz=lc_func(t_z,t0,per,rp,sma,inc,ecc,w,u,limb_dark)
        
        fitlightcurve=m.light_curve(params)
        fitlightcurvep=mp.light_curve(paramsp)
        fitlightcurvez=mz.light_curve(paramsz)
        
        residuals=(fitlightcurve-lc_data_binns[:,b])*10**6.
        chi2=np.nansum(np.abs(residuals/10**6.)**2.)/yerr_binns[b]

        Output=open(SAVEPATH+'UNC_Fits_'+str(int(width))+'/Progress.txt','a')
        Output.write('{0} \n'.format('     -->> Best Fit Params'))
        Output.write('{0} {1} \n'.format('          t0  : ', t0))#,  np.round(t0o[1],5),np.round(t0o[2],5)))                                        
        Output.write('{0} {1} \n'.format('          per : ', per))#, np.round(pero[1],5),np.round(pero[2],5)))                                      
        Output.write('{0} {1} {2} {3} \n'.format('          rp  : ', rp,  np.round(rpo[1],5),np.round(rpo[2],5)))
        Output.write('{0} {1} \n'.format('          a   : ', sma))#, np.round(smao[1],5),np.round(smao[2],5)))                                      
        Output.write('{0} {1} \n'.format('          inc : ', inc))#, np.round(inco[1],5),np.round(inco[2],5)))                                      
        Output.write('{0} {1} {2} {3} \n'.format('          c1  : ', c1,  np.round(c1o[1],5),np.round(c1o[2],5)))
        Output.write('{0} {1} {2} {3} \n'.format('          c2  : ', c2,  np.round(c2o[1],5),np.round(c2o[2],5)))
        Output.write('{0} {1} \n'.format('  chi-squared : ', chi2))
        Output.close()

        plt.figure(1)                                                                                                                               
        plt.plot(t,lc_data_binns[:,b],'.',markersize=10,markeredgecolor='black',markerfacecolor=scal_m.to_rgba(bin_ctr[b]))
        plt.plot(fulltime,fitlightcurvep,'-',color='black')
        plt.ylim(np.nanmin(fitlightcurvep)-0.01,1.01)
        plt.figtext(0.15,0.15,'$\chi^2$ = '+str(chi2))
        plt.figtext(0.55,0.80, str(int(bin_ctr[b]))+' $\AA$',fontsize=25,color=scal_m.to_rgba(bin_ctr[b]))                                         
#        plt.figtext(0.55,0.60, 'White Light', fontsize=25,color='grey')
        plt.savefig(SAVEPATH+'UNC_Fits_'+str(int(width))+'/Fit_Orbit_LC_'+str(int(bin_ctr[b]))+'.png')
        plt.close()

        rpmin=np.max([rpdn,rp-5*rpo[2]])
        rpmax=np.min([rpup,rp+5*rpo[1]])
    
        plt.figure(2)
        corner.corner(runlam,labels=['rp','c1','c2'],truths=[rp,c1,c2],range=([rpmin,rpmax],[c1-5*c1o[2],c1+5*c1o[1]],[c2-5*c2o[2],c2+5*c2o[1]]))
        plt.savefig(SAVEPATH+'UNC_Fits_'+str(int(width))+'/CornerPlot_'+str(int(bin_ctr[b]))+'.png')
        plt.close()

        Output=open(SAVEPATH+'UNC_Fits_'+str(int(width))+'/Progress.txt','a')
        Output.write('{0}\n'.format(' '))
        Output.write('{0} {1} \n'.format('            TIME TO RUN: ', datetime.now() - startTime1))
        Output.write('{0}\n'.format(' '))
        Output.close()

        params=np.array([params.t0,params.per,params.rp,params.a,params.inc,params.u[0],params.u[1]])
        #paramserr=np.array([[t0o[1],t0o[2]],[pero[1],pero[2]],[rpo[1],rpo[2]],[inco[1],inco[2]]])
        paramserr=np.array([[rpo[1],rpo[2]],[c1o[1],c1o[2]],[c2o[1],c2o[2]]])
        save_rp[b]=params[2]
        save_rp_e[b,:]=np.array([rpo[1],rpo[2]])
        save_ld[b,:]=np.array([params[5],params[6]])
        save_ld_e[b,:]=np.array([c1o[1],c1o[2],c2o[1],c2o[2]])

#         np.savez_compressed(SAVEPATH+'UNC_Fits_'+str(int(width))+'/LightCurve_fits_'+str(int(bin_ctr[b]))+'.npz',
#                             results=lamchain,params=params,paramserr=paramserr,
#                             lightcurve_fit=fitlightcurvep,lcfitz=fitlightcurvez,residuals=residuals)
        np.savez_compressed(SAVEPATH+'UNC_Fits_'+str(int(width))+'/LightCurve_fits_'+str(int(bin_ctr[b]))+'.npz',
                            params=params,paramserr=paramserr,
                            lightcurve_fit=fitlightcurvep,lcfitz=fitlightcurvez,residuals=residuals)
        del runlam
        del lamchain
    np.savez_compressed(SAVEPATH+'UNC_Fits_'+str(int(width))+'/FinalResultArrays.npz',
                        rp=save_rp,rp_e=save_rp_e,ld=save_ld,ld_e=save_ld_e)
        

/usr/local/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:14: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/local/anaconda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/local/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 478, in start
  

['/usr/local/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py', '-f', '/Users/ermay/Library/Jupyter/runtime/kernel-591638c7-a0df-46be-a2ef-c115f736a106.json']
3.583893 0.0799592127489 9.24021952675
(72,)
72


IOError: [Errno 2] No such file or directory: '/Volumes/ermay_ext/Magellan_IMACS/ut20170723/SaveData_Hats8_ap5/UNC_Fits_100/Progress.txt'